## Импорт бибилиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import date

import warnings

%pylab inline

from project_7_functions import *

Populating the interactive namespace from numpy and matplotlib


C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Установка параметров и определение констант

In [2]:
from project_7_constants import *

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

##  Загрузка даных

In [3]:
df_kaggle = pd.read_csv(f'{path}auto_ru_2020_09_09.csv',low_memory=False)
df_kaggle_full = pd.read_csv(f'{path}all_auto_ru_09_09_2020.csv',low_memory=False)

test = pd.read_csv(f'{path}test.csv',low_memory=False)

In [4]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

## Приведение к единому виду

In [5]:
%%time 
externdata_test = externdata_test_unification(test)
externdata_train = externdata_train_unification(df_kaggle)
externdata_train_full = externdata_train_unification(df_kaggle_full)

Wall time: 1.6 s


## Объединение test и train

In [6]:
externdata_train['sample'] = 1  # помечаем где у нас трейн
externdata_train_full['sample'] = 1  # помечаем где у нас трейн
externdata_test['sample'] = 0   # помечаем где у нас тест
externdata_test['price'] = np.NaN  # в тесте нет значения price -  пока просто заполняем np.NaN
# объединяем
externdata = externdata_test.append(externdata_train, sort=False).reset_index(drop=True) 
externdata_full = externdata_test.append(externdata_train_full, sort=False).reset_index(drop=True) 

## EDA
Разделим признаки на количественные `num_cols` , категориальные `cat_cols` и бинарные `bin_cols`.   
К бинарным, очевидно, отнесем признаки с двумя уникальными значениями.  
Категориальными признаками будем считать признаки с количеством уникальных значений от 3 до 27 а также очевидно категориальные название модели и марку автомобиля.   
Порогом для количественного признака будет наличие более чем 27 уникальных значений. При этом учитываем, что название модели и марку автомобиля из количественных признаков мы исключили.  


In [7]:
# Посмотри обьекты какого типа находятся теперь в колонках датасета.
# externdata_train_full.apply(lambda x: type(x.iloc[0]))

Определяем кличество уникальных знчений в каждой колонке и относим признак к соответсвующему типу данных (преобразвание `astype(str)` необходимо для избежания `TypeError: ("unhashable type: 'list'", 'occurred at index equipment_dict')`)

In [8]:
num_uniq = externdata.apply(lambda x: x.astype(str).nunique())

In [56]:
num_uniq

bodyType                 102
brand                     12
color                     16
engineDisplacement        56
enginePower              325
equipment_dict         25538
fuelType                   5
mileage                17207
modelDate                 69
model_name               573
numberOfDoors              5
productionDate            72
vehicleTransmission        4
vendor                     2
Владельцы                  4
Владение                 298
ПТС                        3
Привод                     3
Руль                       2
sample                     2
offerprice                 1
price                   6461
dtype: int64

### количественные

### категориальные

### бинарные

In [57]:
bin_cols = externdata.columns[num_uniq==2]
bin_cols

Index(['vendor', 'Руль', 'sample'], dtype='object')

In [ ]:
cat_cols = data.columns[(num_uniq>2) & (num_uniq<28)].append(pd.Index(['model_name','brand'])) 

In [ ]:
num_cols = data.columns[num_uniq>=28].drop(time_col).drop(['model_name','brand','offerprice'])

# ============================

In [33]:
externdata_train_full.equipment_dict.apply(len).nunique()

89

In [53]:
externdata_full.brand.value_counts()

MERCEDES      12192
BMW           11866
VOLKSWAGEN    11543
NISSAN        10165
TOYOTA         8737
AUDI           7505
MITSUBISHI     6776
HYUNDAI        5734
KIA            5630
SKODA          5547
FORD           4797
CHEVROLET      3633
VOLVO          3536
OPEL           3114
RENAULT        2713
HONDA          2328
MAZDA          2311
INFINITI       1911
LEXUS          1888
PEUGEOT        1641
LAND_ROVER     1623
CITROEN        1146
DAEWOO         1041
SUBARU          932
SUZUKI          872
SSANG_YONG      717
PORSCHE         713
CHERY           566
JEEP            483
GREAT_WALL      420
DODGE           418
CHRYSLER        323
CADILLAC        322
JAGUAR          313
MINI            312
GEELY           295
Name: brand, dtype: int64